In [ ]:
from model_utils import configuration as cnf
from plotting import model_plots
from plotting import data_plots
from model_utils import initialisation as init
import sectors
import pyomo.environ as pyo

In [ ]:
# Show network complexity
data_plots.plot_io_network(cnf.DATA)

In [ ]:
# Model setup
model = init.init_model()

sectors.trade.configure_sector(model)
sectors.extraction.configure_sector(model)
sectors.storage.configure_sector(model)

sectors.electricity.configure_sector(model)
sectors.transmission.configure_sector(model)
# # sectors.passenger.configure_sector(model)

sectors.demands.configure_sector(model)

In [ ]:
model.sto_c_charge_limit.pprint()

In [ ]:
tot_cost_expr = model.dem_e_CostTotal + model.elec_e_CostTotal + model.ext_e_CostTotal + model.sto_e_CostTotal + model.trd_e_CostTotal + model.etrans_e_CostTotal

In [ ]:
# tot_cost_expr = model.e_costinv + model.e_costfixedom + model.e_costvar - model.e_revenuevar
model.cost = pyo.Objective(expr=tot_cost_expr, sense=pyo.minimize)
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

opt = pyo.SolverFactory("gurobi", solver_io="python")
opt.options["MIPGap"] = 1e-2
opt.options["Timelimit"] = 1800
try:
    opt_result = opt.solve(model, tee=False)
    print(opt_result)
except ValueError:
    model.write("debug.lp", format="lp", io_options={"symbolic_solver_labels": True})

# Plotting

In [ ]:
model.test.pprint()

In [ ]:
model_plots.plot_flow_fin(model, cnf.DATA, ["elecsupply"], unit="TWh", hist="elecsupply")

In [ ]:
model_plots.plot_flow_fout(model, cnf.DATA, ["elecsupply"], unit="TWh", hist="elecsupply")

In [ ]:
model_plots.plot_group_ctot(model, ["_elec_", "_chp_"], unit="GW")

In [ ]:
model_plots.plot_act(model, "sto_elec_pumpedhydro", unit="TWh")

# Debug
Check your model run :)

In [ ]:
for y_last in model.Years:
    for h in model.Hours:
        tech = "sto_elec_pumpedhydro"
        flow = "elecsupply"
        print(f"", "fin", model.fin[flow, tech, y_last, h].value, "soc", model.a[tech, y_last, h].value, "fout", model.fout[flow, tech, y_last, h].value)

In [ ]:
from pyomo.util import infeasible
infeasible.log_infeasible_constraints(model, log_expression=True, log_variables=True)
infeasible.log_infeasible_bounds(model)
# infeasible.log_close_to_bounds(model)

In [ ]:
import gurobi

model.write("debug.lp", format="lp", io_options={"symbolic_solver_labels": True})
m = gurobi.read("debug.lp")
m.optimize()
m.computeIIS()
m.write("debug_{0}.ilp".format("CHE"))